`` find . -maxdepth 1 -type f | head -100 | xargs cp -t ../trainable_mp3s``

Run this command in train_mp3s file to copy first 100 files into trainable_mp3s

### Opens the input_csv_file and selects the file_rows which are already present in the trainable_mp3s folder and creates the output_csv_file and writes the rows there

In [1]:
import csv

# Specify the input and output file paths
input_csv_file = 'train.csv'
output_csv_file = 'train2.csv'


valid_trainable_mp3s = []
with open(input_csv_file) as csvfile:
    spamreader = csv.DictReader(csvfile, delimiter=',')
    for row in spamreader:
        thing_index = file_names.index(row['id']) if row['id'] in file_names else -1
        if thing_index != -1:
            valid_trainable_mp3s.append(row)

with open(output_csv_file, mode='w', newline='') as csv_file:
    # Create a CSV writer
    csv_writer = csv.DictWriter(csv_file, fieldnames=valid_trainable_mp3s[0].keys())
    csv_writer.writeheader()
    for i in range(len(valid_trainable_mp3s)):
        csv_writer.writerow(valid_trainable_mp3s[i])

NameError: name 'file_names' is not defined

### extracts the filenames of the mp3 files

In [2]:
import os

# Specify the directory path
directory_path = "./trainable_mp3s"

file_names = []

for filename in os.listdir(directory_path):
    # Get the filename without extension
    file_name, file_extension = os.path.splitext(filename)
    file_names.append(file_name)

print(len(file_names))

100


### parse if the wav is for train set or valid set

In [4]:
import csv


def parseTrainValid(filepath, train_val:str):
    data = []
    with open(filepath) as csvfile:
        spamreader = csv.DictReader(csvfile, delimiter=',')
        for row in spamreader:
            if row['split'] == train_val:
                data.append((row['id'], row['sentence']))
                
    return data

In [7]:
trains = parseTrainValid('train2.csv', "train")
# print((trains))

### Converting to wav file and txt file from csv and mp3 file into data directory

In [33]:
import os 
import csv


from multiprocessing.pool import ThreadPool

from sox import Transformer
import tqdm

csvPath = os.path.join("./", "train2.csv")

data_dir = os.path.join("./", "data2")
valid_dir = os.path.join("./", "valid")
# train_dir  = os.path.join("./", "train")
# wav_dir = os.path.join(valid_dir, "wav")
# txt_dir = os.path.join(valid_dir, "txt")

wav_dir = os.path.join(data_dir, "wav")
txt_dir = os.path.join(data_dir, "txt")
mp3_dir = os.path.join("./", "trainable_mp3s")

# print(mp3_dir)

def process(x, wav_dir, txt_dir):
    file_path, text = x
    file_name = os.path.splitext(os.path.basename(file_path))[0]
    text = text.strip()
    with open(os.path.join(txt_dir, file_name + '.txt'), 'w') as f:
        f.write(text)
    audio_path = os.path.join(mp3_dir, f'{file_path}.mp3')
    output_wav_path = os.path.join(wav_dir, file_name + '.wav')

    tfm = Transformer()
    tfm.rate(samplerate=16000)
    tfm.build(
        input_filepath=audio_path,
        output_filepath=output_wav_path
    )

print('Converting mp3 to wav for {}.'.format(csvPath))
with open(csvPath) as csvfile:
    reader = csv.DictReader(csvfile, delimiter=',')
    data = [(row['id'], row['sentence']) for row in reader]
    for i in range(len(data)):
        process(data[i], wav_dir, txt_dir)
        if i ==50:
            break
#     with ThreadPool(1) as pool:
#         list(tqdm.tqdm(pool.imap(process, data), total=len(data)))
        

output_file: ./data2/wav/000005f3362c.wav already exists and will be overwritten on build
output_file: ./data2/wav/000057f5f4a3.wav already exists and will be overwritten on build
output_file: ./data2/wav/0000ab4d937a.wav already exists and will be overwritten on build
output_file: ./data2/wav/0000bd6716f6.wav already exists and will be overwritten on build
output_file: ./data2/wav/0000e711c2b1.wav already exists and will be overwritten on build
output_file: ./data2/wav/0002192aba0b.wav already exists and will be overwritten on build
output_file: ./data2/wav/00037d54a909.wav already exists and will be overwritten on build
output_file: ./data2/wav/0003879c0ff1.wav already exists and will be overwritten on build
output_file: ./data2/wav/000880b04db2.wav already exists and will be overwritten on build
output_file: ./data2/wav/0008d36904cd.wav already exists and will be overwritten on build


Converting mp3 to wav for ./train2.csv.


output_file: ./data2/wav/00096b301cd8.wav already exists and will be overwritten on build
output_file: ./data2/wav/0009b022c8ea.wav already exists and will be overwritten on build
output_file: ./data2/wav/0009dbfd0db3.wav already exists and will be overwritten on build
output_file: ./data2/wav/000a0d4e17e7.wav already exists and will be overwritten on build
output_file: ./data2/wav/000b2b35ede9.wav already exists and will be overwritten on build
output_file: ./data2/wav/000b78fc1e98.wav already exists and will be overwritten on build
output_file: ./data2/wav/000bdc393875.wav already exists and will be overwritten on build
output_file: ./data2/wav/000ceddf1350.wav already exists and will be overwritten on build
output_file: ./data2/wav/000d5f179e10.wav already exists and will be overwritten on build
output_file: ./data2/wav/000d6860da9c.wav already exists and will be overwritten on build
output_file: ./data2/wav/000dfa480dd6.wav already exists and will be overwritten on build
output_fil

### Creating manifest.json file

In [2]:
# data[:50]

In [29]:
file_paths = []
with open(csvPath) as csvfile:
    reader = csv.reader(csvfile)
    heading = next(reader)
    for row in reader:
        file_paths.append(row[0])

In [31]:
file_paths = file_paths[:50]

In [36]:
import json
from pathlib import Path

# data_path = os.path.join(data_dir, "wav")
valid_path = os.path.abspath(valid_dir)
data_path = os.path.abspath(data_dir)

output_name =  "manifest.json"
# output_path = Path(data_dir) / output_name
output_path = Path(valid_dir) / output_name

manifest = {
    'root_path': valid_path,
    'samples': []
}



for file_path in tqdm.tqdm(file_paths):
    wav_path = os.path.join(f'wav/{file_path}.wav')
    transcript_path = os.path.join(f'txt/{file_path}.txt')
    manifest['samples'].append({
        'wav_path': wav_path,
        'transcript_path': transcript_path
    })

output_path.write_text(json.dumps(manifest), encoding='utf8')

100%|███████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 202427.80it/s]


4137

In [17]:
def _duration_file_path(path):
    return path, sox.file_info.duration(path)


def order_and_prune_files(
        file_paths,
        min_duration,
        max_duration,
        num_workers):
    print("Gathering durations...")
    with Pool(processes=num_workers) as p:
        duration_file_paths = list(tqdm(p.imap(_duration_file_path, file_paths), total=len(file_paths)))
    print("Sorting manifests...")
    if min_duration and max_duration:
        print("Pruning manifests between %d and %d seconds" % (min_duration, max_duration))
        duration_file_paths = [(path, duration) for path, duration in duration_file_paths if
                               min_duration <= duration <= max_duration]

    total_duration = sum([x[1] for x in duration_file_paths])
    print(f"Total duration of split: {total_duration:.4f}s")
    return [x[0] for x in duration_file_paths]  # Remove durations




### Select the data from train.csv and write in train2.csv based on trainable_mp3s folder

In [ ]:
# import
from bnunicodenormalizer import Normalizer 
from pprint import pprint
# initialize
bnorm=Normalizer()
# normalize
word = 'াটোবাকো'
result=bnorm(word)
print(f"Non-norm:{word}; Norm:{result['normalized']}")
print("--------------------------------------------------")
pprint(result)

In [ ]:
# initialize without english (default)
norm=Normalizer()
print("without english:",norm("ASD123")["normalized"])
# --> returns None
norm=Normalizer(allow_english=True)
print("with english:",norm("ASD123")["normalized"])

In [ ]:
transcript = list(filter(None, [labels_map.get(x) for x in list(result['normalized'])]))

In [8]:
import csv

# Specify the input and output file paths
input_csv_file = 'train.csv'
output_csv_file = 'train2.csv'

# Open the input CSV file in read mode
with open(input_csv_file, mode='r', newline='') as input_file:
    # Create a CSV reader
    csv_reader = csv.reader(input_file)

    # Read the header row
    header = next(csv_reader)

    # Read and store the first 1000 rows
    rows_to_write = []
    for i, row in enumerate(csv_reader):
#         if i < 1000:
        rows_to_write.extend(row[1])
#             print(row[1])
#         else:
#             break

# Open the output CSV file in write mode
# with open(output_csv_file, mode='w', newline='') as output_file:
#     # Create a CSV writer
#     csv_writer = csv.writer(output_file)

#     # Write the header row
#     csv_writer.writerow(header)

#     # Write the selected rows
#     csv_writer.writerows(rows_to_write)

print(f"Spliced first 1000 rows from '{input_csv_file}' and wrote to '{output_csv_file}'.")


Spliced first 1000 rows from 'train.csv' and wrote to 'train2.csv'.


In [13]:
len(rows_to_write)
myset = set(rows_to_write)
print(myset)

{'ফ', ' ', 'া', 'ৈ', '৷', 'ঋ', "'", 'খ', 'ঙ', 'উ', '…', '‚', 'ধ', 'ূ', 'য়', ':', ';', 'ক', '?', '–', 'ঠ', 'ও', 'গ', '“', 'থ', 'শ', 'ঁ', 'ে', '—', '॥', 'ৃ', 'ড', 'ঢ়', 'ভ', 'অ', 'ন', 'ৗ', '!', 'ি', 'ঢ', 'ব', 'চ', '”', 'ঘ', 'ষ', '/', '-', 'ঐ', 'স', 'হ', 'আ', 'ত', 'ছ', 'ট', 'ঊ', 'র', 'প', 'ং', '.', '়', 'ঞ', 'ৰ', 'জ', 'ঈ', 'ঃ', 'ণ', '’', 'ৌ', 'এ', 'ল', '"', '‘', 'ু', ',', 'ৎ', '্', 'য', 'দ', 'ঝ', '৵', 'ো', 'ই', 'ঔ', 'ড়', '।', 'ম', 'ী'}


In [15]:
myset

{' ',
 '!',
 '"',
 "'",
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '?',
 '।',
 '॥',
 'ঁ',
 'ং',
 'ঃ',
 'অ',
 'আ',
 'ই',
 'ঈ',
 'উ',
 'ঊ',
 'ঋ',
 'এ',
 'ঐ',
 'ও',
 'ঔ',
 'ক',
 'খ',
 'গ',
 'ঘ',
 'ঙ',
 'চ',
 'ছ',
 'জ',
 'ঝ',
 'ঞ',
 'ট',
 'ঠ',
 'ড',
 'ঢ',
 'ণ',
 'ত',
 'থ',
 'দ',
 'ধ',
 'ন',
 'প',
 'ফ',
 'ব',
 'ভ',
 'ম',
 'য',
 'র',
 'ল',
 'শ',
 'ষ',
 'স',
 'হ',
 '়',
 'া',
 'ি',
 'ী',
 'ু',
 'ূ',
 'ৃ',
 'ে',
 'ৈ',
 'ো',
 'ৌ',
 '্',
 'ৎ',
 'ৗ',
 'ড়',
 'ঢ়',
 'য়',
 'ৰ',
 '৵',
 '৷',
 '–',
 '—',
 '‘',
 '’',
 '‚',
 '“',
 '”',
 '…'}

In [18]:
labels = [
 ' ',
 '!',
 '"',
 "'",
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '?',
 '।',
 '॥',
 'ঁ',
 'ং',
 'ঃ',
 'অ',
 'আ',
 'ই',
 'ঈ',
 'উ',
 'ঊ',
 'ঋ',
 'এ',
 'ঐ',
 'ও',
 'ঔ',
 'ক',
 'খ',
 'গ',
 'ঘ',
 'ঙ',
 'চ',
 'ছ',
 'জ',
 'ঝ',
 'ঞ',
 'ট',
 'ঠ',
 'ড',
 'ঢ',
 'ণ',
 'ত',
 'থ',
 'দ',
 'ধ',
 'ন',
 'প',
 'ফ',
 'ব',
 'ভ',
 'ম',
 'য',
 'র',
 'ল',
 'শ',
 'ষ',
 'স',
 'হ',
 '়',
 'া',
 'ি',
 'ী',
 'ু',
 'ূ',
 'ৃ',
 'ে',
 'ৈ',
 'ো',
 'ৌ',
 '্',
 'ৎ',
 'ৗ',
 'ড়',
 'ঢ়',
 'য়',
 'ৰ',
 '৵',
 '৷',
 '–',
 '—',
 '‘',
 '’',
 '‚',
 '“',
 '”',
 '…'
]

In [26]:
len(labels)

87

In [23]:
a = '–'
b = '—'

In [25]:
print(ord(a), ord(b))

8211 8212
